In [59]:
from io import BytesIO
from IPython.display import clear_output, Image, display
from PIL import Image as Img
import cv2
import dlib
import numpy as np
from scipy.spatial import distance as dist

In [60]:
class_dlib_68_points = "classificadores/shape_predictor_68_face_landmarks.dat"
class_lib = dlib.shape_predictor(class_dlib_68_points)
detect_face = dlib.get_frontal_face_detector()

In [61]:
def set_img(img):
    imagem = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    imagem = cv2.resize(img, (800,600))
    return imagem

def show_video(frame):
    img = Img.fromarray(frame, "RGB")
    buffer = BytesIO()
    img.save(buffer, format="JPEG")
    display(Image(data=buffer.getvalue()))
    clear_output(wait=True)

def points_marcos_faces(img):
    rectangles = detect_face(img, 1)

    if len(rectangles) == 0:
        return None

    marcos = []
    for rect in rectangles:
        marcos.append(np.matrix([[p.x, p.y] for p in class_lib(img, rect).parts()]))

    return marcos    

def anothetion_marcos_faces(img, marcos_faces):
    for marco in marcos_faces:
        for idx, ponto in enumerate(marco):
            center = (ponto[0, 0], ponto[0, 1])
            cv2.circle(img, center, 3, color=(255, 255, 0), thickness=-1)
    return img

In [45]:
webcam = cv2.VideoCapture(0)

try:
    while(True):
        video_ok, frame = webcam.read()

        if video_ok:
            frame = set_img(frame)
            if points := points_marcos_faces(frame):                
                frame = anothetion_marcos_faces(frame, points)
            show_video(frame)

except KeyboardInterrupt:
    webcam.release()
    print("Break")

[ WARN:0@1619.030] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@1619.031] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


Break


In [64]:
FACE = list(range(17, 68))
FACE_COMPLET = list(range(0, 68))
LIPS = list(range(48, 61))
EYEBROW_RIGHT = list(range(17, 22))
EYEBROW_LEFT = list(range(22, 27))
EYE_RIGHT = list(range(36,42))
EYE_LEFT = list(range(42,48))
NOSE = list(range(27,35))
JAW = list(range(0,17))

In [65]:
def aspect_reason_mouth(points_mouth):
    a = dist.euclidean(points_mouth[3], points_mouth[9])
    b = dist.euclidean(points_mouth[2], points_mouth[10])
    c = dist.euclidean(points_mouth[4], points_mouth[8])
    d = dist.euclidean(points_mouth[0], points_mouth[6])
    return (a + b + c)/(3.0 * d)

In [66]:
webcam = cv2.VideoCapture(0)

try:
    while(True):
        video_ok, frame = webcam.read()

        if video_ok:
            frame = set_img(frame)
            points = points_marcos_faces(frame)
            if points is not None:
                frame = anothetion_marcos_faces(frame, points)
                points = points[0]
                ar = aspect_reason_mouth(points[LIPS])
                ar = round(ar,3)
                coord = tuple(points[JAW][0].A1.reshape(1, -1)[0])
                coord = (coord[0], coord[1] + 20)
                cv2.putText(frame, "Aspecto Razao " + str(ar), coord, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 2)

            show_video(frame)

except KeyboardInterrupt:
    webcam.release()
    print("Break")

ValueError: Input vector should be 1-D.